In [1]:
pip install tflearn -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gky16\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
os with open('C:\\Users\\gky16\\OneDrive\\Desktop\\deque\\medical chat data') as f:
    data = json.load(f)

SyntaxError: invalid syntax (2487138866.py, line 1)

In [7]:
import os # import the os module
with open("C:\\Users\\gky16\\OneDrive\\Desktop\deque\\medical chat data\\intents.json") as f: # open the file with the with statement
    data = json.load(f) # load the data as a JSON object


In [8]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])


  if intent["tag"] not in labels:
    labels.append(intent["tag"])  

words = [stemmer.stem(w.lower()) for w in words if  w not in "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

training = np.array(training)
output = np.array(output)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training, output, n_epoch = 1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 23999  | total loss: 0.07214 | time: 0.052s
| Adam | epoch: 1000 | loss: 0.07214 - acc: 0.9967 -- iter: 184/188
Training Step: 24000  | total loss: 0.06499 | time: 0.060s
| Adam | epoch: 1000 | loss: 0.06499 - acc: 0.9970 -- iter: 188/188
--
INFO:tensorflow:C:\Users\gky16\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return np.array(bag)

def chat():
  print("\033[1;31m Start talking with the bot!, Type quit when you want to quit")
  while True:
    inp = input("\033[1;31m You: ")
    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      
      if tg['tag'] == tag:
        
        responses = tg['responses']

        response_list = nltk.sent_tokenize(str(responses[0]))
        colors = [31, 32, 33, 34, 35]

        for i in range(len(response_list)):
            color_index = i % len(colors)
            print(f'\033[1;{colors[color_index]}m {response_list[i]:<12s}\n')

In [ ]:
chat()

 Start talking with the bot!, Type quit when you want to quit
 You: hello
 1)Ice the bruise with an ice pack wrapped in a towel.

 2)Leave it in place for 10 to 20 minutes.

 3)Repeat several times a day for a day or two as needed.

 4)Compress the bruised area if it is swelling, using an elastic bandage.

 You: hii
 1)Ice the bruise with an ice pack wrapped in a towel.

 2)Leave it in place for 10 to 20 minutes.

 3)Repeat several times a day for a day or two as needed.

 4)Compress the bruised area if it is swelling, using an elastic bandage.

 You: gaurav
 1)Ice the bruise with an ice pack wrapped in a towel.

 2)Leave it in place for 10 to 20 minutes.

 3)Repeat several times a day for a day or two as needed.

 4)Compress the bruised area if it is swelling, using an elastic bandage.

 You: child
 1)Ice the bruise with an ice pack wrapped in a towel.

 2)Leave it in place for 10 to 20 minutes.

 3)Repeat several times a day for a day or two as needed.

 4)Compress the bruised area i